# MolGrid Demo - Jupyter Notebook

This notebook demonstrates the MolGrid interactive molecule grid visualization component.

MolGrid provides an interactive grid for browsing, searching, and selecting molecules with features including:

- Pagination for large datasets
- Text search by molecular properties
- SMARTS substructure filtering
- Selection with export to SMILES/CSV
- Info tooltips with molecular data
- Full DataFrame integration

## Setup

First, import the required packages.

In [ ]:
import cnotebook
from cnotebook import MolGrid, molgrid
from openeye import oechem
import pandas as pd
import oepandas as oepd

## Basic Usage

### Creating a Simple Grid from Molecules

The simplest way to use MolGrid is to pass a list of OpenEye molecule objects.

In [ ]:
# Create some test molecules
smiles_data = [
    ("CCO", "Ethanol"),
    ("CC(=O)O", "Acetic Acid"),
    ("c1ccccc1", "Benzene"),
    ("CC(=O)Nc1ccc(O)cc1", "Acetaminophen"),
    ("CC(C)Cc1ccc(C(C)C(=O)O)cc1", "Ibuprofen"),
    ("CN1C=NC2=C1C(=O)N(C(=O)N2C)C", "Caffeine"),
    ("CC(=O)OC1=CC=CC=C1C(=O)O", "Aspirin"),
    ("CN1CCC[C@H]1c2cccnc2", "Nicotine"),
]

molecules = []
for smi, name in smiles_data:
    mol = oechem.OEGraphMol()
    oechem.OESmilesToMol(mol, smi)
    mol.SetTitle(name)
    molecules.append(mol)

# Create and display a basic grid
grid = MolGrid(molecules)
grid.display()

### Using the Convenience Function

The `molgrid()` function provides a simpler interface.

In [ ]:
grid = molgrid(molecules, n_items_per_page=4)
grid.display()

## Customizing the Display

### Image Size and Format

Customize the molecule image dimensions and format.

In [ ]:
grid = MolGrid(
    molecules,
    width=150,
    height=150,
    image_format="svg",  # or "png"
    n_items_per_page=8,
)
grid.display()

### Title Display

Control how molecule titles are displayed using the `title_field` parameter.

In [ ]:
# Show titles (default behavior)
grid = MolGrid(molecules, title_field="Title")
grid.display()

In [ ]:
# Hide titles
grid = MolGrid(molecules, title_field=None)
grid.display()

## Search and Filtering

MolGrid provides two search modes:

1. **Properties Mode**: Text search across molecule titles and properties
2. **SMARTS Mode**: Substructure filtering using SMARTS patterns

Use the toggle switch in the toolbar to switch between modes.

Try searching for:
- Properties mode: "acid" or "caffeine"
- SMARTS mode: "c1ccccc1" (aromatic ring) or "[OH]" (hydroxyl group)

In [ ]:
grid = MolGrid(molecules, n_items_per_page=8)
grid.display()

## Selection

### Enabling Selection

Selection is enabled by default. Click on molecules or use the checkbox to select them.

In [ ]:
grid = MolGrid(molecules, select=True, name="selection-demo")
grid.display()

### Retrieving Selected Molecules

After making selections in the grid above, run the cell below to retrieve the selected molecules.

In [ ]:
# Get selected molecule objects
selected_mols = grid.get_selection()
print(f"Selected {len(selected_mols)} molecules:")
for mol in selected_mols:
    print(f"  - {mol.GetTitle()}")

# Get selected indices
indices = grid.get_selection_indices()
print(f"\nSelected indices: {indices}")

### Selection Actions Menu

The "..." menu in the toolbar provides selection actions:

- **Select All**: Select all visible (filtered) molecules
- **Clear Selection**: Deselect all molecules
- **Invert Selection**: Toggle selection state of all visible molecules
- **Copy to Clipboard**: Copy selected molecules as CSV
- **Save to SMILES**: Download selected molecules as a .smi file
- **Save to CSV**: Download selected molecules as a .csv file

## Info Button and Tooltips

Each molecule cell has an info button ("i") in the top-right corner that displays molecular data.

- **Hover** over the "i" to see the tooltip
- **Click** the "i" to pin the tooltip open (useful for comparing molecules)
- **Click again** to unpin

The info button shows:
- Index (always)
- Title (if available)
- Any columns specified via the `data` parameter

In [ ]:
# Add SD data to molecules for display in tooltips
for mol in molecules:
    oechem.OESetSDData(mol, "MW", str(oechem.OECalculateMolecularWeight(mol)))
    oechem.OESetSDData(mol, "Formula", oechem.OEMolecularFormula(mol))

# Display specific data fields in the info tooltip
grid = MolGrid(molecules, data=["MW", "Formula"])
grid.display()

### Disabling the Info Button

Set `information=False` to hide the info button.

In [ ]:
grid = MolGrid(molecules, information=False)
grid.display()

## DataFrame Integration

MolGrid integrates seamlessly with Pandas DataFrames containing molecule columns.

### Creating a Grid from a DataFrame

In [ ]:
# Create a DataFrame with molecules and properties
df = pd.DataFrame({
    "Name": ["Ethanol", "Benzene", "Acetaminophen", "Ibuprofen", "Caffeine", "Aspirin"],
    "SMILES": ["CCO", "c1ccccc1", "CC(=O)Nc1ccc(O)cc1", 
               "CC(C)Cc1ccc(C(C)C(=O)O)cc1", "CN1C=NC2=C1C(=O)N(C(=O)N2C)C",
               "CC(=O)OC1=CC=CC=C1C(=O)O"],
    "Category": ["Alcohol", "Aromatic", "Analgesic", "NSAID", "Stimulant", "NSAID"],
    "MW": [46.07, 78.11, 151.16, 206.28, 194.19, 180.16],
})

# Convert SMILES to molecules
mols = []
for _, row in df.iterrows():
    mol = oechem.OEGraphMol()
    oechem.OESmilesToMol(mol, row["SMILES"])
    mol.SetTitle(row["Name"])
    mols.append(mol)

df["Molecule"] = mols
df["Molecule"] = df["Molecule"].astype(oepd.MoleculeDtype())

df

In [ ]:
# Create grid from DataFrame
grid = MolGrid(
    df["Molecule"].tolist(),
    dataframe=df,
    mol_col="Molecule",
)
grid.display()

### Auto-Detection of Search Fields

When using a DataFrame, MolGrid automatically detects string columns for text search.

In [ ]:
# The Name and Category columns are automatically searchable
print(f"Search fields: {grid.search_fields}")
print(f"Info fields: {grid.information_fields}")

### Specifying Data Fields

Use the `data` parameter to control which columns appear in the info tooltip.

In [ ]:
# Show only specific columns in the info tooltip
grid = MolGrid(
    df["Molecule"].tolist(),
    dataframe=df,
    mol_col="Molecule",
    data=["Category", "MW"],  # Only show these in tooltip
)
grid.display()

### Using the DataFrame Accessor

For convenience, you can create a MolGrid directly from a DataFrame column.

In [ ]:
# Create grid using the .chem accessor
grid = df["Molecule"].chem.molgrid()
grid.display()

In [ ]:
# Create grid from DataFrame with options
grid = df.chem.molgrid(
    mol_col="Molecule",
    title_field="Name",
    tooltip_fields=["Category", "MW"],
)
grid.display()

## Complete Example

Here is a complete example combining multiple features.

In [ ]:
# Create a comprehensive grid with all features
grid = MolGrid(
    df["Molecule"].tolist(),
    dataframe=df,
    mol_col="Molecule",
    title_field="Name",
    tooltip_fields=["SMILES"],
    data=["Category", "MW"],
    n_items_per_page=6,
    width=180,
    height=180,
    image_format="svg",
    select=True,
    information=True,
    name="complete-demo",
)
grid.display()

## API Reference

### MolGrid Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `mols` | Iterable | (required) | OpenEye molecule objects |
| `dataframe` | DataFrame | None | Optional DataFrame with molecule data |
| `mol_col` | str | None | Column name containing molecules |
| `title_field` | str | "Title" | Field to display as title (None to hide) |
| `tooltip_fields` | List[str] | None | Fields for hover tooltip |
| `n_items_per_page` | int | 24 | Molecules per page |
| `width` | int | 200 | Image width in pixels |
| `height` | int | 200 | Image height in pixels |
| `atom_label_font_scale` | float | 1.5 | Scale factor for atom labels |
| `image_format` | str | "svg" | Image format ("svg" or "png") |
| `select` | bool | True | Enable selection checkboxes |
| `information` | bool | True | Enable info button |
| `data` | str/List[str] | None | Columns for info tooltip (auto-detects if None) |
| `search_fields` | List[str] | None | Fields for text search |
| `name` | str | None | Grid identifier |

### MolGrid Methods

| Method | Returns | Description |
|--------|---------|-------------|
| `display()` | HTML | Display the grid in the notebook |
| `to_html()` | str | Generate HTML representation |
| `get_selection()` | List[OEMol] | Get selected molecule objects |
| `get_selection_indices()` | List[int] | Get indices of selected molecules |